# Particionamento de Dados com Apache Iceberg

O particionamento é uma técnica fundamental para otimizar consultas em grandes volumes de dados. O Apache Iceberg oferece recursos avançados como **Particionamento Oculto** e **Evolução de Particionamento**.

## Conceitos Principais:

### 1. Particionamento Oculto (Hidden Partitioning)
- O Iceberg abstrai a complexidade do particionamento
- Usuário define apenas a coluna e função de transformação (ex: `year(data_venda)`)
- Não precisa incluir colunas derivadas nas consultas
- Simplifica queries e evita erros

### 2. Evolução de Particionamento (Partition Evolution)
- Permite alterar estratégia de particionamento sem reescrever dados
- Novos dados usam novo esquema, dados antigos mantêm esquema original
- Adapta-se ao crescimento e mudanças nos padrões de consulta

## Setup do Ambiente

In [2]:
# Para o Spark se estiver rodando
try:
    spark.stop()
except:
    pass

# Configuração do Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark-3.3.0-bin-hadoop3"

import findspark
findspark.init('/opt/spark-3.3.0-bin-hadoop3')

from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col

# Sessão Spark com configuração explícita
spark = SparkSession.builder \
    .appName("IcebergPartitioning") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.hadoop_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.hadoop_catalog.type", "hadoop") \
    .config("spark.sql.catalog.hadoop_catalog.warehouse", "file:///home/tavares/warehouse") \
    .config("spark.sql.default.catalog", "hadoop_catalog") \
    .config("spark.hadoop.fs.defaultFS", "file:///") \
    .getOrCreate()

print(f"Warehouse configurado para: {spark.conf.get('spark.sql.catalog.hadoop_catalog.warehouse')}")

Warehouse configurado para: file:///home/tavares/warehouse


## 1. Particionamento por Ano

Vamos criar uma tabela particionada por ano usando a função `year(data_venda)`. Isso permite que consultas filtradas por ano leiam apenas os arquivos relevantes.

In [3]:
# Criar tabela particionada por ano da data de venda
spark.sql("""
    CREATE TABLE IF NOT EXISTS hadoop_catalog.default.vendas_partitioned (
        id INT,
        produto STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
    PARTITIONED BY (year(data_venda))
""")

DataFrame[]

### Inserindo Dados de Diferentes Anos

Vamos inserir dados de 2022, 2023 e 2024 para demonstrar como o Iceberg organiza automaticamente os dados em partições por ano.

In [4]:
# Inserir dados de diferentes anos
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas_partitioned VALUES
    (1, 'Produto A', 10, 15.5, DATE('2022-01-15')),
    (2, 'Produto B', 5, 22.0, DATE('2022-06-20')),
    (3, 'Produto C', 8, 30.0, DATE('2023-03-10')),
    (4, 'Produto D', 12, 25.0, DATE('2023-08-25')),
    (5, 'Produto E', 7, 18.5, DATE('2024-02-14'))
""")

DataFrame[]

### Visualizando os Dados

Observe que consultamos diretamente a coluna `data_venda` sem precisar especificar a partição.

In [5]:
# Visualizar dados
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_partitioned ORDER BY data_venda").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 15.5|2022-01-15|
|  2|Produto B|         5| 22.0|2022-06-20|
|  3|Produto C|         8| 30.0|2023-03-10|
|  4|Produto D|        12| 25.0|2023-08-25|
|  5|Produto E|         7| 18.5|2024-02-14|
+---+---------+----------+-----+----------+



### Visualizando as Partições

O Iceberg cria automaticamente partições baseadas no ano. Cada partição contém estatísticas úteis para otimização.

In [6]:
# Ver informações sobre partições
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_partitioned.partitions").show()

+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|partition|spec_id|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|     last_updated_at|last_updated_snapshot_id|
+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|     {54}|      0|           1|         1|                         1393|                           0|                         0|                           0|                         0|2025-11-03 01:03:...|     1286999254922226430|
|     {52}|      0|           2|         1|                         1397

### Consulta Otimizada por Partição

Esta consulta lerá apenas os arquivos da partição de 2023, demonstrando o **partition pruning**.

In [7]:
# Consulta que aproveita particionamento (apenas dados de 2023)
spark.sql("""
    SELECT * FROM hadoop_catalog.default.vendas_partitioned 
    WHERE year(data_venda) = 2023
    ORDER BY data_venda
""").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  3|Produto C|         8| 30.0|2023-03-10|
|  4|Produto D|        12| 25.0|2023-08-25|
+---+---------+----------+-----+----------+



### Visualizando Arquivos por Partição

Podemos ver como os arquivos estão organizados fisicamente por partição.

In [8]:
# Ver arquivos da tabela particionada
spark.sql("SELECT file_path, partition FROM hadoop_catalog.default.vendas_partitioned.files").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------+---------+
|file_path                                                                                                                                     |partition|
+----------------------------------------------------------------------------------------------------------------------------------------------+---------+
|file:/home/tavares/warehouse/default/vendas_partitioned/data/data_venda_year=2023/00005-5-9059b484-a4c6-4aa7-ba7d-b4ca57677dde-0-00001.parquet|{53}     |
|file:/home/tavares/warehouse/default/vendas_partitioned/data/data_venda_year=2022/00044-6-9059b484-a4c6-4aa7-ba7d-b4ca57677dde-0-00001.parquet|{52}     |
|file:/home/tavares/warehouse/default/vendas_partitioned/data/data_venda_year=2024/00061-7-9059b484-a4c6-4aa7-ba7d-b4ca57677dde-0-00001.parquet|{54}     |
+---------------------------------------------------------------------

## 2. Particionamento por Dias

Para dados com alta granularidade temporal, podemos usar `days(data_venda)` que particiona por dia desde a época Unix.

In [9]:
# Criar tabela com particionamento por dias (evita conflito year/month)
spark.sql("""
    CREATE TABLE IF NOT EXISTS hadoop_catalog.default.vendas_daily (
        id INT,
        produto STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
    PARTITIONED BY (days(data_venda))
""")

DataFrame[]

### Inserindo Dados na Tabela Diária

Cada data única criará uma partição separada.

In [10]:
# Inserir dados na tabela com particionamento diário
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas_daily VALUES
    (1, 'Produto A', 10, 15.5, DATE('2023-01-15')),
    (2, 'Produto B', 5, 22.0, DATE('2023-01-20')),
    (3, 'Produto C', 8, 30.0, DATE('2023-02-10')),
    (4, 'Produto D', 12, 25.0, DATE('2023-02-25')),
    (5, 'Produto E', 7, 18.5, DATE('2023-03-14'))
""")

DataFrame[]

### Visualizando Partições Diárias

Observe que cada data única cria uma partição com número de dias desde 1970-01-01.

In [11]:
# Ver partições da tabela diária
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_daily.partitions").show()

+------------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|   partition|spec_id|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|     last_updated_at|last_updated_snapshot_id|
+------------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|{2023-02-10}|      0|           1|         1|                         1391|                           0|                         0|                           0|                         0|2025-11-03 01:04:...|     2485143199939475997|
|{2023-03-14}|      0|           1|         1|              

## 3. Particionamento por Bucket

O particionamento por bucket distribui dados uniformemente usando uma função hash. Útil para balanceamento de carga e joins eficientes.

In [12]:
# Criar tabela com particionamento por bucket (alternativa para múltiplas dimensões)
spark.sql("""
    CREATE TABLE IF NOT EXISTS hadoop_catalog.default.vendas_bucket (
        id INT,
        produto STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
    PARTITIONED BY (bucket(4, id))
""")

DataFrame[]

### Inserindo Dados com Bucket

Os dados serão distribuídos em 4 buckets baseados no hash do campo `id`.

In [13]:
# Inserir dados na tabela com bucket
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas_bucket VALUES
    (1, 'Produto A', 10, 15.5, DATE('2023-01-15')),
    (2, 'Produto B', 5, 22.0, DATE('2023-01-20')),
    (3, 'Produto C', 8, 30.0, DATE('2023-02-10')),
    (4, 'Produto D', 12, 25.0, DATE('2023-02-25')),
    (5, 'Produto E', 7, 18.5, DATE('2023-03-14'))
""")

DataFrame[]

### Visualizando Partições por Bucket

Cada bucket (0-3) contém dados distribuídos uniformemente pelo hash do ID.

In [14]:
# Ver partições por bucket
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_bucket.partitions").show()

+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|partition|spec_id|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|     last_updated_at|last_updated_snapshot_id|
+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|      {0}|      0|           2|         1|                         1398|                           0|                         0|                           0|                         0|2025-11-03 01:04:...|     1726455918655621727|
|      {2}|      0|           1|         1|                         1393

## 4. Particionamento por Categoria

Particionamento por valores categóricos é útil quando temos consultas frequentes filtradas por categoria específica.

In [15]:
# Exemplo de particionamento por categoria de produto
spark.sql("""
    CREATE TABLE IF NOT EXISTS hadoop_catalog.default.vendas_categoria (
        id INT,
        produto STRING,
        categoria STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
    PARTITIONED BY (categoria)
""")

DataFrame[]

### Inserindo Dados com Categorias

Cada categoria criará uma partição separada, otimizando consultas por categoria.

In [16]:
# Inserir dados com categorias
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas_categoria VALUES
    (1, 'Notebook', 'Eletrônicos', 2, 2500.0, DATE('2023-01-15')),
    (2, 'Mesa', 'Móveis', 1, 800.0, DATE('2023-01-20')),
    (3, 'Smartphone', 'Eletrônicos', 3, 1200.0, DATE('2023-02-10')),
    (4, 'Cadeira', 'Móveis', 4, 300.0, DATE('2023-02-25')),
    (5, 'Tablet', 'Eletrônicos', 1, 600.0, DATE('2023-03-14'))
""")

DataFrame[]

### Consulta Otimizada por Categoria

Esta consulta lerá apenas os arquivos da partição 'Eletrônicos', ignorando dados de outras categorias.

In [17]:
# Consultar apenas eletrônicos
spark.sql("""
    SELECT * FROM hadoop_catalog.default.vendas_categoria 
    WHERE categoria = 'Eletrônicos'
    ORDER BY data_venda
""").show()

+---+----------+-----------+----------+------+----------+
| id|   produto|  categoria|quantidade| preco|data_venda|
+---+----------+-----------+----------+------+----------+
|  1|  Notebook|Eletrônicos|         2|2500.0|2023-01-15|
|  3|Smartphone|Eletrônicos|         3|1200.0|2023-02-10|
|  5|    Tablet|Eletrônicos|         1| 600.0|2023-03-14|
+---+----------+-----------+----------+------+----------+



### Visualizando Partições por Categoria

Podemos ver as partições criadas para cada categoria única.

In [18]:
# Ver partições por categoria
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_categoria.partitions").show()

+-------------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|    partition|spec_id|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|     last_updated_at|last_updated_snapshot_id|
+-------------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|     {Móveis}|      0|           2|         1|                         1686|                           0|                         0|                           0|                         0|2025-11-03 01:04:...|     9072823416610609881|
|{Eletrônicos}|      0|           3|         1|         